In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from tqdm import tqdm
import re

# Как чистился датасет

In [2]:
b = pd.read_csv('data/GoodReads_100k_books.csv')
print(b.isna().sum())

author              0
bookformat       3228
desc             6772
genre           10467
img              3045
isbn            14482
isbn13          11435
link                0
pages               0
rating              0
reviews             0
title               1
totalratings        0
dtype: int64


In [3]:
b_clean = b.dropna(subset=['desc', 'genre'])
print(b_clean.isna().sum())

author              0
bookformat       1431
desc                0
genre               0
img               634
isbn            11344
isbn13           8566
link                0
pages               0
rating              0
reviews             0
title               0
totalratings        0
dtype: int64


In [4]:
b_clean['auth_title'] = b_clean['author'] + ' ' + b_clean['title']
b_clean = b_clean.drop_duplicates(subset=['auth_title'])

C:\Windows\Temp\ipykernel_10616\85333368.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b_clean['auth_title'] = b_clean['author'] + ' ' + b_clean['title']


In [5]:
b_clean = b_clean.drop(['bookformat', 'reviews', 'pages', 'auth_title'], axis=1)

In [6]:
b_clean.sort_values('author')

,author,desc,genre,img,isbn,isbn13,link,rating,title,totalratings
67606,&. Ruprecht Vandenhoeck,"English summary: Want to find quick, short and...","History,European Literature,German Literature,...",https://i.gr-assets.com/images/S/compressed.ph...,3525320116,9.78353E+12,https://goodreads.com/book/show/14692896-der-g...,5.00,Der Grosse Ploetz: Die Chronik Zur Weltgeschichte,1
35626,'Captain' Robert Brown,"There will come a time...,when the freedom is ...","Science Fiction,Steampunk,Fantasy,Science Fict...",https://i.gr-assets.com/images/S/compressed.ph...,NaN,NaN,https://goodreads.com/book/show/12477387-the-w...,3.60,The Wrath of Fate: Book 1 of The Airship Pirat...,163
48952,"'Cato',Michael Foot,Peter Howard,Frank Owen","A polemic against Chamberlain, MacDonald, and ...","History,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,141180986,9.78E+12,https://goodreads.com/book/show/1327921.Guilty...,3.84,Guilty Men,50
99396,'Nathan Burgoine,"Kieran Quinn is a bit telepathic, a little psy...","Romance,M M Romance,Fantasy,Paranormal,Fantasy...",https://i.gr-assets.com/images/S/compressed.ph...,1602829535,9.7816E+12,https://goodreads.com/book/show/17290850-light,4.17,Light,349
65098,'Sharkey' Ward,This volume is a controversial account of what...,"Aviation,Military,Military History,History,War...",https://i.gr-assets.com/images/S/compressed.ph...,304355429,9.7803E+12,https://goodreads.com/book/show/144216.Sea_Har...,4.30,Sea Harrier Over The Falklands,185
...,...,...,...,...,...,...,...,...,...,...
88899,"è—¤å´Ž éƒ½,Miyako Fujisaki,Shungiku Nakamura",æ–‡åº«ç‰ˆ2å†Šã®åˆæœ¬ã«æãä¸‹ã‚ã—æ¼«ç”»...,"Sequential Art,Manga,Manga,Yaoi,Novels,Light N...",https://i.gr-assets.com/images/S/compressed.ph...,4041005884,9.78404E+12,https://goodreads.com/book/show/16192969-3-sek...,4.31,ä¸–ç•Œä¸€åˆæ‹ ã€œæ¨ªæ¾¤éš†å²ã®å ´åˆ3ã€œ [...,340
42582,"è‘£å•Ÿç« ,æŽæ™ºæµ·","æ™‚é–“æ˜¯1997å¹´ä»¥å¾Œ,1998è‡³1999å¹´ï¼Œæ™šæœŸ...","Cultural,China",https://i.gr-assets.com/images/S/compressed.ph...,9570838221,9.79E+12,https://goodreads.com/book/show/12962841,3.33,å¤¢è¯éŒ„,12
44298,"è‡ªç”±è¡Œèµ°,Tá»± Do HÃ nh Táº©u","TÃ¬nh yÃªu thá»© ba - Tá»± do hÃ nh táº©u,ã€Šç...","Asian Literature,Chinese Literature,Romance,Cu...",https://i.gr-assets.com/images/S/compressed.ph...,NaN,NaN,https://goodreads.com/book/show/13059208-t-nh-...,3.89,TÃ¬nh yÃªu thá»© ba,65
32469,"é¡¾æ¼«,Gu Man",BÃªn nhau trá»n Ä‘á»i lÃ cuá»‘n sÃ¡ch náº±m...,"Romance,Cultural,China,Fiction,Contemporary,No...",https://i.gr-assets.com/images/S/compressed.ph...,NaN,NaN,https://goodreads.com/book/show/12267726-b-n-n...,3.70,BÃªn nhau trá»n Ä‘á»i,1155


In [7]:
pattern = r'[^a-zA-Z0-9 \]\[+$#*/?!&\-:;,\.\'\")(]'

b_clean = b_clean[b_clean['title'].apply(lambda x: not bool(re.search(pattern, x)))]
b_clean = b_clean[b_clean['author'].apply(lambda x: not bool(re.search(pattern, x)))]
b_clean = b_clean[b_clean['desc'].apply(lambda x: not bool(re.search(pattern, x)))]

In [8]:
b_clean

,author,desc,genre,img,isbn,isbn13,link,rating,title,totalratings
0,Laurence M. Hauptman,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,3.52,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,4.51,Fashion Sourcebook 1920s,41
2,Andy Anderson,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,4.15,Hungary 56,26
3,Carlotta R. Anderson,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,3.83,All-American Anarchist: Joseph A. Labadie and ...,6
6,Jeffrey Pfeffer,"""Competitive Advantage Through People"" explore...","Business,Leadership,Business,Management,Romanc...",https://i.gr-assets.com/images/S/compressed.ph...,087584717X,9.78E+12,https://goodreads.com/book/show/1001092.Compet...,3.65,Competitive Advantage Through People: Unleashi...,20
...,...,...,...,...,...,...,...,...,...,...
99993,Philip Hoare,"A startling new book, his most personal to dat...","Nonfiction,Environment,Nature,Travel,Science,A...",https://i.gr-assets.com/images/S/compressed.ph...,7412118,9.78001E+12,https://goodreads.com/book/show/17320067-the-s...,3.77,The Sea Inside,497
99994,Sarah Lean,Sometimes when things are broken you can't fix...,"Animals,Childrens,Middle Grade,Animals,Horses,...",https://i.gr-assets.com/images/S/compressed.ph...,7455054,9.78001E+12,https://goodreads.com/book/show/17320079-a-hor...,3.97,A Horse for Angel,535
99997,C. John Miller,In these warm reflections on his own growth as...,"Christianity,Evangelism,Christian,Religion,The...",https://i.gr-assets.com/images/S/compressed.ph...,875523919,9.78088E+12,https://goodreads.com/book/show/173203.A_Faith...,4.27,A Faith Worth Sharing: A Lifetime of Conversat...,122
99998,Albert Marrin,"John Brown is a man of many legacies, from her...","Nonfiction,History,Biography,Military History,...",https://i.gr-assets.com/images/S/compressed.ph...,307981533,9.78031E+12,https://goodreads.com/book/show/17320669-a-vol...,3.63,A Volcano Beneath the Snow: John Brown's War A...,156


In [9]:
b_clean.to_csv('GoodReads_clean.csv')

In [10]:
b_clean.loc[1,:]

author                           Charlotte Fiell,Emmanuelle Dirix
desc            Fashion Sourcebook - 1920s is the first book i...
genre                   Couture,Fashion,Historical,Art,Nonfiction
img             https://i.gr-assets.com/images/S/compressed.ph...
isbn                                                   1906863482
isbn13                                                   9.78E+12
link            https://goodreads.com/book/show/10010552-fashi...
rating                                                       4.51
title                                    Fashion Sourcebook 1920s
totalratings                                                   41
Name: 1, dtype: object

In [11]:
b_clean.duplicated().sum()

0

In [21]:
b_clean.groupby('auth_title')['desc'].apply(lambda x: len(x))

auth_title
&. Ruprecht VandenhoeckDer Grosse Ploetz: Die Chronik Zur Weltgeschichte                                                                   1
'Captain' Robert BrownThe Wrath of Fate: Book 1 of The Airship Pirate Chronicles                                                           1
'Cato',Michael Foot,Peter Howard,Frank OwenGuilty Men                                                                                      1
'Nathan BurgoineLight                                                                                                                      1
'Sharkey' WardSea Harrier Over The Falklands                                                                                               1
                                                                                                                                          ..
è—¤å´Ž éƒ½,Miyako Fujisaki,Shungiku Nakamuraä¸–ç•Œä¸€åˆæ‹ ã€œæ¨ªæ¾¤éš†å²ã®å ´åˆã€œ [Sekaiichi Hatsukoi: Yokozawa Takafumi no Baai]    1
è‘

In [11]:
b_clean.isna().sum()

author              0
bookformat       1431
desc                0
genre               0
img               634
isbn            11344
isbn13           8566
link                0
pages               0
rating              0
reviews             0
title               0
totalratings        0
dtype: int64

In [22]:
b_clean.dropna(subset=['genre'])

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
5,Jeffrey Pfeffer,Hardcover,Why is common sense so uncommon when it comes ...,"Business,Leadership,Romance,Historical Romance...",https://i.gr-assets.com/images/S/compressed.ph...,875848419,9.78E+12,https://goodreads.com/book/show/1001090.The_Hu...,368,3.73,7,The Human Equation: Building Profits by Puttin...,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99993,Philip Hoare,Paperback,"A startling new book, his most personal to dat...","Nonfiction,Environment,Nature,Travel,Science,A...",https://i.gr-assets.com/images/S/compressed.ph...,7412118,9.78001E+12,https://goodreads.com/book/show/17320067-the-s...,350,3.77,74,The Sea Inside,497
99994,Sarah Lean,Paperback,Sometimes when things are broken you can't fix...,"Animals,Childrens,Middle Grade,Animals,Horses,...",https://i.gr-assets.com/images/S/compressed.ph...,7455054,9.78001E+12,https://goodreads.com/book/show/17320079-a-hor...,288,3.97,94,A Horse for Angel,535
99997,C. John Miller,Paperback,In these warm reflections on his own growth as...,"Christianity,Evangelism,Christian,Religion,The...",https://i.gr-assets.com/images/S/compressed.ph...,875523919,9.78088E+12,https://goodreads.com/book/show/173203.A_Faith...,160,4.27,20,A Faith Worth Sharing: A Lifetime of Conversat...,122
99998,Albert Marrin,Hardcover,"John Brown is a man of many legacies, from her...","Nonfiction,History,Biography,Military History,...",https://i.gr-assets.com/images/S/compressed.ph...,307981533,9.78031E+12,https://goodreads.com/book/show/17320669-a-vol...,256,3.63,51,A Volcano Beneath the Snow: John Brown's War A...,156


In [8]:
b_clean['auth_title'].value_counts()

Laurence M. Hauptman Between Two Fires: American Indians in the Civil War       1
David  Diop,Simon Mpondo,Frank Jones Hammer Blows and Other Writings            1
Rachel Isadora,Jacob Grimm The Twelve Dancing Princesses                        1
Verna Aardema,Bryna Waldman Anansi Finds a Fool: An Ashanti Tale                1
Ulli Beier Yoruba Myths                                                         1
                                                                               ..
Hester Burton,Victor G. Ambrus Time of Trial                                    1
Elizabeth  Lister,A.E. Lister The Beach House                                   1
Unknown,Edward E. Hunt Sir Orfeo                                                1
Nicole Dennis Fire Moon Dance                                                   1
Marc E. Fitch Paranormal Nation: Why America Needs Ghosts, UFOs, and Bigfoot    1
Name: auth_title, Length: 85446, dtype: int64

In [4]:
b['img'][45]

'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1180124846l/1001466.jpg'

In [29]:
b['desc'].dropna().apply(lambda x: len(x))

0         192
1         843
2         187
3        1125
4         527
         ... 
99995    1265
99996     486
99997     196
99998     685
99999     880
Name: desc, Length: 93228, dtype: int64

In [11]:
b['desc'].isna().sum()

6772

In [4]:
id = None

# Пасер

In [26]:
soup = BeautifulSoup(data.text)

In [31]:
soup.find(name='div', attrs={'class' : 'isbn'}).text.split()[1]

'978-5-386-14975-8'

In [11]:
authors = [_.text for _ in soup.find_all(name='a', attrs={'data-event-label':'author'})]
authors

['Хазин Михаил Леонидович', 'Щеглов Сергей']

In [8]:
title = soup.find(name='div', attrs={'id':'product-about'}).h2.text[19:-1]
title

'Кризис и Власть. Том 2. Люди Власти'

In [9]:
annotation = soup.find(name='div', attrs={'id':'fullannotation'}).p.text  # для тех случаев когда есть fullannotation и smallannotation
annotation

'В вопросах Власти есть два ответа - всем известный и правильный. Любой представитель элиты заинтересован в том, чтобы сохранить в тайне структуру своей группировки и ее подлинные цели. Но помимо внутренних связей существуют еще и публичные действия, совершаемые членами группировок в ходе борьбы за те или иные ресурсы.   С помощью теории Власти, изложенной в "Лестнице в небо", Михаил Хазин и Сергей Щеглов анализируют масштабные исторические и политические события, чтобы показать, сколько коллективных интересов, мотивов и проблем скрывается за известными каждому "фактами". За что поплатился директор МВФ Стросс-Кан? Мог ли Сталин проиграть борьбу за власть? Кто на самом деле стоит за смертью Кеннеди и почему абсолютно все, даже его сторонники, поддержали официальную версию об убийце-одиночке? Почему Рузвельт, затянувший Великую депрессию, стал единственным президентом в истории США, избранным на третий срок, а Никсон, который вывел американскую экономику из кризиса, - единственным презид

In [18]:
genres = soup.find(name='div', attrs={'id':'thermometer-books'}).find_all(name='span', attrs={'itemprop':'title'})
genres = [_.text for _ in genres]
if genres[0] != 'Книги':
    pass  # break
genres = genres[1:]
genres

['Нехудожественная литература', 'Публицистика', 'Политика']

In [12]:
rating = float(soup.find(name='div', attrs={'id':'rate'}).text)
rating

9.75

In [15]:
rated = soup.find(name='div', attrs={'id':'product-rating-marks-label'}).text
rated = int(rated.split()[-1][:-1])
rated

8

In [32]:
data_2 = requests.get('https://www.labirint.ru/books/938733')
soup_2 = BeautifulSoup(data_2.text)

In [33]:
genres = soup_2.find(name='div', attrs={'id':'thermometer-books'}).find_all(name='span', attrs={'itemprop':'title'})
genres = [_.text for _ in genres]
if genres[0] != 'Книги':
    pass  # break
genres = genres[1:]
genres

['Комиксы, Манга, Артбуки', 'Манга']

In [21]:
authors = [_.text for _ in soup_2.find_all(name='a', attrs={'data-event-label':'author'})]
authors

['Ди Няньмяо']

In [22]:
title = soup_2.find(name='div', attrs={'id':'product-about'}).h2.text[19:-1]
title

'Магмелл синего моря. Том 1'

In [24]:
annotation = soup_2.find(name='div', attrs={'id':'product-about'}).find_all('p')[-1].text  # для тех случаев когда есть fullannotation и smallannotation
annotation

'Тридцать шесть лет назад, с началом Эпохи Чудес, весь мир содрогнулся от того, что произошло посреди безбрежного Тихого океана. Из ниоткуда появился новый материк, который назвали Магмеллом, или Священным материком! Всё на нём казалось таким загадочным и дарило столько надежд: фантастические пейзажи, ценные ископаемые, невиданные животные и растения! Человечество будто вернулось в эпоху великих открытий, и для изучения новых земель на Священный материк стали отправляться исследователи. Но вскоре понадобились представители ещё одной профессии - искатели…'

In [27]:
rating = float(soup_2.find(name='div', attrs={'id':'rate'}).text)
rating

0.0

In [28]:
rated = soup_2.find(name='div', attrs={'id':'product-rating-marks-label'}).text
rated = int(rated.split()[-1][:-1])
rated

0

In [29]:
data_3 = requests.get('https://www.labirint.ru/books/981211/')
soup_3 = BeautifulSoup(data_3.text)

In [30]:
try:
    genres = soup_3.find(name='div', attrs={'id':'thermometer-books'}).find_all(name='span', attrs={'itemprop':'title'})
except:
    pass  # continue
genres = [_.text for _ in genres]
if genres[0] != 'Книги':
    pass  # continue
genres = genres[1:]
genres

AttributeError: 'NoneType' object has no attribute 'find_all'

In [2]:
df = pd.DataFrame(columns=['isbn', 'title', 'authors', 'publisher', 'genres', 'rating', 'people_rated', 'annotation'])
df

,isbn,title,authors,publisher,genres,rating,people_rated,annotation


In [5]:
if len(df) == 0:
    print('s')

s


In [9]:
for i in tqdm(range(1, 100, 1)):
    data = requests.get('https://www.labirint.ru/books/' + str(i))
    soup = BeautifulSoup(data.text)
    object = {}
    
    try:
        genres = soup.find(name='div', attrs={'id':'thermometer-books'}).find_all(name='span', attrs={'itemprop':'title'})
    except:
        # print(i)
        continue
    genres = [_.text for _ in genres]
    if genres[0] != 'Книги':
        continue
    if genres[1] == 'Художественная литература':
        genres = genres[1:]
        object['genres'] = '/'.join(genres)
        # object['genres'] = genres
    elif genres[2] == 'Детская художественная литература':
        genres = genres[2:]
        object['genres'] = '/'.join(genres)
        # object['genres'] = genres
    else:
        continue
    
    try:
        title = soup.find(name='div', attrs={'id':'product-about'}).h2.text[19:-1]
    except:
        continue
    object['title'] = title
    
    
    annotation = soup.find(name='div', attrs={'id':'product-about'}).find_all('p')[-1].text
    # if len(annotation) < 200:
    #     continue
    object['annotation'] = annotation
    
    authors = [_.text for _ in soup.find_all(name='a', attrs={'data-event-label':'author'})]
    if authors:
        object['authors'] = '/'.join(authors)
        
    publisher = soup.find(name='a', attrs={'data-event-label':'publisher'})
    if publisher:
        object['publisher'] = publisher
    
    
    rating = float(soup.find(name='div', attrs={'id':'rate'}).text)
    object['rating'] = rating
    
    rated = soup.find(name='div', attrs={'id':'product-rating-marks-label'}).text
    rated = int(rated.split()[-1][:-1])
    object['people_rated'] = rated
    
    isbn = soup.find(name='div', attrs={'class' : 'isbn'}).text.split()[1]
    object['isbn'] = isbn
    
    df = pd.concat([df, pd.DataFrame(data=object, index=[i])])
    # df = df.append(object, ignore_index=True)

100%|██████████| 99/99 [01:51<00:00,  1.12s/it]


In [10]:
df

,isbn,title,authors,publisher,genres,rating,people_rated,annotation
31,5-7811-0106-3,Эпитафия: Иначе не выжить,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.0,1,Что может помешать нажать на спусковой крючок?...
32,5-7811-0107-1,Эпитафия. Гробовщик,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.0,1,"Тот, кто поднялся из ничтожества, больше всего..."
33,5-7811-0105-5,Эпитафия: Кровавый источник,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,0.0,0,Крупный уральский город становится ареной ожес...


In [133]:
df.to_csv('books.csv')

In [5]:
df.to_parquet('books.parquet')

In [50]:
! pip install pyarrow

In [2]:
q = pd.read_parquet('books.parquet')
q

,isbn,title,authors,publisher,genres,rating,people_rated,annotation
31,5-7811-0106-3,Эпитафия: Иначе не выжить,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.00,1,Что может помешать нажать на спусковой крючок?...
32,5-7811-0107-1,Эпитафия. Гробовщик,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.00,1,"Тот, кто поднялся из ничтожества, больше всего..."
33,5-7811-0105-5,Эпитафия: Кровавый источник,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,0.00,0,Крупный уральский город становится ареной ожес...
151,5-8301-0055-X,Петер Каменцинд,Гессе Герман,Амфора,Художественная литература/Классическая проза/К...,10.00,1,В Первый том Собрания сочинений Германа Гессе...
157,5-89178-123-9,Пьесы. Сценарии. Рассказы,Володин Александр Моисеевич,У-Фактория,Художественная литература/Драматургия/Отечеств...,10.00,3,"В книгу вошли пьесы, сценарии, рассказы, стихи..."
...,...,...,...,...,...,...,...,...
168408,978-5-8189-1128-1,"""Мама для мамонтенка"" и другие истории",Непомнящая Дина,Гелеос,Детская художественная литература/Проза для де...,9.73,11,Старый друг - лучше новых двух! Самые популярн...
168429,978-5-9539-2152-7,W.I.T.C.H. Истории о чародейках: Все о Ирме,None,Эгмонт,Детская художественная литература/Проза для де...,9.69,16,"Ирма рассказывает все! О том, как строила глаз..."
168460,978-5-9781-0087-7,Тимур и его команда,Гайдар Аркадий Петрович,Самовар,Детская художественная литература/Проза для де...,8.74,54,Серия рекомендована Департаментом общего средн...
168460,978-5-9781-0087-7,Тимур и его команда,Гайдар Аркадий Петрович,Самовар,Детская художественная литература/Проза для де...,8.74,54,Серия рекомендована Департаментом общего средн...


In [10]:
q.duplicated().sum()
q.drop_duplicates()

,isbn,title,authors,publisher,genres,rating,people_rated,annotation
31,5-7811-0106-3,Эпитафия: Иначе не выжить,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.00,1,Что может помешать нажать на спусковой крючок?...
32,5-7811-0107-1,Эпитафия. Гробовщик,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.00,1,"Тот, кто поднялся из ничтожества, больше всего..."
33,5-7811-0105-5,Эпитафия: Кровавый источник,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,0.00,0,Крупный уральский город становится ареной ожес...
151,5-8301-0055-X,Петер Каменцинд,Гессе Герман,Амфора,Художественная литература/Классическая проза/К...,10.00,1,В Первый том Собрания сочинений Германа Гессе...
157,5-89178-123-9,Пьесы. Сценарии. Рассказы,Володин Александр Моисеевич,У-Фактория,Художественная литература/Драматургия/Отечеств...,10.00,3,"В книгу вошли пьесы, сценарии, рассказы, стихи..."
...,...,...,...,...,...,...,...,...
168407,978-5-8189-1127-4,"""Крошка енот"" и другие истории",None,Гелеос,Детская художественная литература/Проза для де...,9.60,10,Старый друг - лучше новых двух! Самые популярн...
168408,978-5-8189-1128-1,"""Мама для мамонтенка"" и другие истории",Непомнящая Дина,Гелеос,Детская художественная литература/Проза для де...,9.73,11,Старый друг - лучше новых двух! Самые популярн...
168429,978-5-9539-2152-7,W.I.T.C.H. Истории о чародейках: Все о Ирме,None,Эгмонт,Детская художественная литература/Проза для де...,9.69,16,"Ирма рассказывает все! О том, как строила глаз..."
168460,978-5-9781-0087-7,Тимур и его команда,Гайдар Аркадий Петрович,Самовар,Детская художественная литература/Проза для де...,8.74,54,Серия рекомендована Департаментом общего средн...


In [7]:
q['annotation'].apply(lambda x: len(x))

31      374
32      327
33      378
151     182
157     480
       ... 
2651    239
2716      6
2792    151
2832     62
2834     46
Name: annotation, Length: 172, dtype: int64

In [ ]:
q.query

In [11]:
q

,isbn,title,authors,publisher,genres,rating,people_rated,annotation


In [14]:
pd.concat([q, df])

,isbn,title,authors,publisher,genres,rating,people_rated,annotation
31,5-7811-0106-3,Эпитафия: Иначе не выжить,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.0,1,Что может помешать нажать на спусковой крючок?...
32,5-7811-0107-1,Эпитафия. Гробовщик,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.0,1,"Тот, кто поднялся из ничтожества, больше всего..."
33,5-7811-0105-5,Эпитафия: Кровавый источник,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,0.0,0,Крупный уральский город становится ареной ожес...


In [15]:
df.to_parquet('books.parquet')

In [6]:
q.iloc[0, :].annotation

'Варить самогон из сахара, фруктов, ягод и других продуктов в домашних условиях научит вас эта брошюра. Вы узнаете о доступных любому хозяину и хозяйке рецептах различных водок - от пшеничной до анисовой. Даже секреты приготовления коньяка раскроет вам эта книга.??'

In [2]:
dat = pd.read_csv('train.csv')
dat

,bid,is_aggregate,source,chapter_path,summary_path,book_id,summary_id,content,summary,chapter,chapter_length,summary_name,summary_url,summary_text,summary_analysis,summary_length,analysis_length
0,27681,True,cliffnotes,all_chapterized_books/27681-chapters/chapters_...,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapters 1-2,chapters 1-2,NaN,"{""name"": ""Chapters 1-2"", ""url"": ""https://web.a...","\n ""Mine ear is open, and my heart prepared:\...",6471.0,Chapters 1-2,https://web.archive.org/web/20201101053205/htt...,"Before any characters appear, the time and geo...",These two chapters introduce the reader to the...,388.0,473.0
1,27681,False,cliffnotes,all_chapterized_books/27681-chapters/03.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 3,chapter 3,NaN,"{""name"": ""Chapter 3"", ""url"": ""https://web.arch...","\n ""Before these fields were shorn and tilled...",3132.0,Chapter 3,https://web.archive.org/web/20201101053205/htt...,In another part of the forest by the river a f...,This chapter introduces the other three main a...,198.0,149.0
2,27681,False,cliffnotes,all_chapterized_books/27681-chapters/04.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 4,chapter 4,NaN,"{""name"": ""Chapter 4"", ""url"": ""https://web.arch...","\n ""Well, go thy way: thou shalt not from thi...",3075.0,Chapter 4,https://web.archive.org/web/20201101053205/htt...,When the mounted party from Fort Howard approa...,Since this chapter is mostly one of surface ac...,319.0,75.0
3,27681,False,cliffnotes,all_chapterized_books/27681-chapters/05.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 5,chapter 5,NaN,"{""name"": ""Chapter 5"", ""url"": ""https://web.arch...","\n ""In such a night\n Di...",3268.0,Chapter 5,https://web.archive.org/web/20201101053205/htt...,"The pursuit of Magua is unsuccessful, but Hawk...",Here the reader encounters the first bloodshed...,329.0,156.0
4,27681,False,cliffnotes,all_chapterized_books/27681-chapters/06.txt,finished_summaries/cliffnotes/The Last of the ...,The Last of the Mohicans.chapter 6,chapter 6,NaN,"{""name"": ""Chapter 6"", ""url"": ""https://web.arch...","\n ""Those strains that once did sweet in Zion...",3873.0,Chapter 6,https://web.archive.org/web/20201101053205/htt...,Heyward and the girls are uneasy and Gamut is ...,This chapter shows Cooper in his most inventiv...,321.0,128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/52.txt,finished_summaries/pinkmonkey/Portrait of a La...,Portrait of a Lady.chapter 52,chapter 52,NaN,"{""name"": ""Chapter 52"", ""url"": ""https://web.arc...",\nThere was a train for Turin and Paris that e...,3730.0,Chapter 52,https://web.archive.org/web/20180820033733/htt...,"Before Isabel leaves Rome, she goes to see Pan...",Notes The visit to the convent reveals the nex...,336.0,269.0
9596,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/53.txt,finished_summaries/pinkmonkey/Portrait of a La...,Portrait of a Lady.chapter 53,chapter 53,NaN,"{""name"": ""Chapter 53"", ""url"": ""https://web.arc...","\nIt was not with surprise, it was with a feel...",2875.0,Chapter 53,https://web.archive.org/web/20180820033733/htt...,Isabel is greeted by Henrietta Stackpole at Ch...,Notes This chapter gives the reader a bit of a...,186.0,91.0
9597,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/54.txt,finished_summaries/pinkmonkey/Portrait of a La...,Portrait of a Lady.chapter 54,chapter 54,NaN,"{""name"": ""Chapter 54"", ""url"": ""https://web.arc...",\nIsabel's arrival at Gardencourt on this seco...,3580.0,Chapter 54,https://web.archive.org/web/20180820033733/htt...,Isabel arrives at Gardencourt. The house is ve...,"Notes The two conversations in this chapter, o...",435.0,137.0
9598,2833,False,pinkmonkey,all_chapterized_books/2833-chapters/55.txt,finished_s

In [8]:
dat[['book_id', 'summary_id', 'chapter', 'summary_name', 'summary_text', 'summary_analysis']].head(1)

,book_id,summary_id,chapter,summary_name,summary_text,summary_analysis
0,The Last of the Mohicans.chapters 1-2,chapters 1-2,"\n ""Mine ear is open, and my heart prepared:\...",Chapters 1-2,"Before any characters appear, the time and geo...",These two chapters introduce the reader to the...


In [10]:
dat.loc[0, 'summary_analysis']

"These two chapters introduce the reader to the historical and natural settings and are indicative of the extent to which this book, as a historical novel, relates its fictional characters to real history. Only here at the beginning and later at mid-novel will the action coincide in detail with actual events, though the historic war is always somewhere in the near distance. These chapters also present four of the main fictional characters and one secondary one, all of whom will merit our concern henceforth. Major Heyward is the gallant romantic hero, but unlike most sentimental romances where for each hero there is one heroine, here there are two, Alice and Cora, blonde and brunette. And it is immediately apparent that the old tradition of weak-blonde-strong-brunette contrast is at work, stereotyping the fair Alice and dark Cora. These three are rather predictable types which both simplify and stultify the writer's efforts with them. Magua's stealthy eyes and abrupt, furtive actions ma

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

__author__ = 'ipetrash'


from multiprocessing.dummy import Pool as ThreadPool
import time

from bs4 import BeautifulSoup
import requests


def go(url):
    rs = requests.get(url)
    root = BeautifulSoup(rs.content, 'html.parser')
    return root.select_one('.user-details > a').text.strip()


urls = [
    'https://ru.stackoverflow.com/questions/962400', 'https://ru.stackoverflow.com/questions/962311',
    'https://ru.stackoverflow.com/questions/962128', 'https://ru.stackoverflow.com/questions/962396',
    'https://ru.stackoverflow.com/questions/962349'
]

t = time.time()
result = [go(url) for url in urls]
print(result)
print('Elapsed {:.3f} secs'.format(time.time() - t))
# ['Streletz', 'Kromster', 'Stepan Kasyanenko', 'Kromster', 'JamesJGoodwin']
# Elapsed 6.030 secs

print()

t = time.time()
pool = ThreadPool()
result = pool.map(go, urls)
print(result)
print('Elapsed {:.3f} secs'.format(time.time() - t))
# ['Streletz', 'Kromster', 'Stepan Kasyanenko', 'Kromster', 'JamesJGoodwin']
# Elapsed 3.203 secs

In [17]:
df = pd.DataFrame(columns=['isbn', 'title', 'authors', 'publisher', 'genres', 'rating', 'people_rated', 'annotation'])
df

,isbn,title,authors,publisher,genres,rating,people_rated,annotation


In [1]:
from multiprocessing.dummy import Pool as ThreadPool

In [19]:
pool = ThreadPool()
pool.map(go, range(1, 100))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [14]:
def go(id):
    global df
    data = requests.get('https://www.labirint.ru/books/' + str(id))
    soup = BeautifulSoup(data.text)
    object = {}

    try:
        genres = soup.find(name='div', attrs={'id':'thermometer-books'}).find_all(name='span', attrs={'itemprop':'title'})
    except:
        # print(i)
        return
    genres = [_.text for _ in genres]
    if genres[0] != 'Книги':
        return
    if genres[1] == 'Художественная литература':
        genres = genres[1:]
        object['genres'] = '/'.join(genres)
        # object['genres'] = genres
    elif genres[2] == 'Детская художественная литература':
        genres = genres[2:]
        object['genres'] = '/'.join(genres)
        # object['genres'] = genres
    else:
        return

    try:
        title = soup.find(name='div', attrs={'id':'product-about'}).h2.text[19:-1]
    except:
        return
    object['title'] = title


    annotation = soup.find(name='div', attrs={'id':'product-about'}).find_all('p')[-1].text
    # if len(annotation) < 200:
    #     continue
    object['annotation'] = annotation

    authors = [_.text for _ in soup.find_all(name='a', attrs={'data-event-label':'author'})]
    if authors:
        object['authors'] = '/'.join(authors)

    publisher = soup.find(name='a', attrs={'data-event-label':'publisher'})
    if publisher:
        object['publisher'] = publisher


    rating = float(soup.find(name='div', attrs={'id':'rate'}).text)
    object['rating'] = rating

    rated = soup.find(name='div', attrs={'id':'product-rating-marks-label'}).text
    rated = int(rated.split()[-1][:-1])
    object['people_rated'] = rated

    isbn = soup.find(name='div', attrs={'class' : 'isbn'}).text.split()[1]
    object['isbn'] = isbn

    df = pd.concat([df, pd.DataFrame(data=object, index=[id])])
    # df = df.append(object, ignore_index=True)

In [20]:
df

,isbn,title,authors,publisher,genres,rating,people_rated,annotation
31,5-7811-0106-3,Эпитафия: Иначе не выжить,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.0,1,Что может помешать нажать на спусковой крючок?...
32,5-7811-0107-1,Эпитафия. Гробовщик,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,10.0,1,"Тот, кто поднялся из ничтожества, больше всего..."
33,5-7811-0105-5,Эпитафия: Кровавый источник,Ковалев Анатолий Евгеньевич,Лабиринт,Художественная литература/Детективы/Современны...,0.0,0,Крупный уральский город становится ареной ожес...
